In [ ]:
pip install requests

In [ ]:
pip install selenium

In [ ]:
pip install beautifulsoup4

In [1]:
# import necessary modules
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from selenium import webdriver
from tqdm import tqdm
import time
import datetime
import random
import urllib.request 
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import os
# browser.quit() # close the whole browser
# browser.close() # close the tab

In [ ]:
#URL = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html'
URL = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96'
#URL = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=24'
'''
page = requests.get(URL)
driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
driver.get(URL)
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup.prettify())

product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
print(len(product_cards))

i = 0
for product_card in product_cards:
    i += 1
    product_link = product_card.find('a')['href']
    image = product_card.find('div', {'class' : "BrowseProductCard-imageWrap"})
    image_link = image.find('img')['src']

    print(f"Image link here: {image_link}\n")
    print(f"Link to product {i} here: {product_link}\n")
'''

In [5]:
def get_product_links_from_mainpage(URL):
    link_list = []
    page = requests.get(URL)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(random.randint(3,3))
    product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
    print(len(product_cards))
    i = 0
    for product_card in product_cards:
        try:
            i += 1
            #get product link
            product_link = product_card.find('a')['href']
            link_list.append(product_link)
            
            #get image link
            #image = product_card.find('div', {'class' : "BrowseProductCard-imageWrap"})
            #image_link = image.find('img')['src']
            
            #print info
            #print(f"Image link here: {image_link}\n")
            #print(f"Link to product {i} here: {product_link}\n")
        except:
            pass

    driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    return link_list

def get_info_from_productlink(product_link):  #get the first image
    info = dict()
    page = requests.get(product_link)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(product_link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(random.randint(3,3))

    #price_block = soup.find('div', {"class" : "BasePriceBlock BasePriceBlock--highlight"})
    #price_block = soup.find('div', {"class" : "BasePriceBlock"})
    price_block = soup.find('span', {"class" : "notranslate"})
    price = price_block.text
    info['price'] = price
    #print(price)   
    
    image = soup.find('div', {'class' : "ImageComponent ImageComponent--overlay"})
    image_link = image.find('img')['src']
    info['img_url'] = image_link
    
    #print(f"Image link: {image_link}\n")
    #print(f"Link to product: {product_link}\n")

    driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    return info
    

In [6]:
get_info_from_productlink('https://www.wayfair.com/furniture/pdp/three-posts-brinson-genuine-leather-chesterfield-95-rolled-arm-sofa-w001243560.html?piid=1221870775')

{'price': '$1,759.99',
 'img_url': 'https://secure.img1-fg.wfcdn.com/im/77377094/resize-h800-w800%5Ecompr-r85/7888/78886170/Brinson+Genuine+Leather+Chesterfield+95%2522+Rolled+Arm+Sofa.jpg'}

In [7]:
def save_data(product_info_list, i):
    result = [[product[0], product[1]['price'], product[1]['img_url']] for product in product_info_list]
    df = pd.DataFrame(result)
    now = str(datetime.datetime.now()).replace(" ","").replace(":","").replace("-","").split(".")[0][4:][:-2]
    df.to_csv(f'sofa_info_{str(i)}.csv', index=False, header=['item_no', 'price', 'img_url'])
    
def save_image_from_url(x, category):    
    item_no = x.name+1
    filename = f'./{category}/{item_no}.jpeg'
    url = x['img_url']
    try:
        urllib.request.urlretrieve(url, filename)
        return filename
    except:
        print('failed')
        return 'failed'
    
def save_images_from_csv(df):
    df['filename'] = df.progress_apply(lambda x: save_image_from_url(x, category = 'sofa'), axis=1)
    return df

In [8]:
get_info_from_productlink("https://www.wayfair.com/furniture/pdp/eichholtz-taylor-sofa-eitz3452.html?piid=")

{'price': '$8,399.99',
 'img_url': 'https://secure.img1-fg.wfcdn.com/im/43833910/resize-h800-w800%5Ecompr-r85/1019/101965755/Taylor+Sofa.jpg'}

In [ ]:
product_info = []
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='
base_url = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage='
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='
#for page in tqdm(range(1,201)):

#data from pages 1 - 4
for page in tqdm(range(1,5)):
    url = base_url + str(page)
    link_list = get_product_links_from_mainpage(url)  # latter will work even this is None
    print(f"Number of products on Page {page}: ", len(link_list))
    i = 5779
    for product_link in tqdm(link_list):
        i += 1
        #item_no = f"{page}_{i}"
        item_no = f"{i}"
        try:
            info = get_info_from_productlink(product_link)
            product_info.append((item_no,info))     
        except:
            print(f"Failed. Product link: {product_link}")
            pass
save_data(product_info,1)
product_info

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

96
Number of products on Page 1:  96



 20%|████████████████▏                                                                 | 19/96 [04:48<18:24, 14.35s/it]

In [ ]:
df = pd.read_csv('sofa_info_1.csv')
print(df.shape)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df_new = save_images_from_csv(df)
df_new.to_csv('sofa.csv', index=False)

In [ ]:
df_new